# Newton Sketch: A Linear-time Optimization Algorithm with Linear-Quadratic Convergence

## Newton Sketch Algorithm

In [ ]:
import numpy as np
#import fjlt
from scipy.optimize import minimize
import time

def Newton_Sketch(f, grad, Hess_sr, x0, m, SketchType='SubSamp'):
    # algorithm parameters
    tol = 1e-6
    maxiter = 1000

    
    #initialize quantities and note sizes of quantities
    d = x0.size
    H_sr = Hess_sr(x0);
    n = H_sr.shape[0]
        
    #solve the newton-sketch problem
    dx = 100 #dummy value to start iterating
    xs = np.zeros((x0.size, maxiter+1))
    xs[:,0] = x0
    t0 = time.time()
    times = np.zeros((1, maxiter+1))
    numiter = 0
    x = x0
    for i in range(maxiter):
        #print(numiter)
        
        
        
        #define S and B = S*H_sr
        H_sr = Hess_sr(x);
        g = grad(x)
        if SketchType is 'Gaussian':
            S = np.random.randn(m, n)
            B = S @ H_sr
        if SketchType is 'SubSamp':
            ind = np.random.permutation(n)[:m]
            B = H_sr[ind, :]
            
        #find the approximate newton direction
        
        
        subprob = lambda v: g.T@v + 0.5*np.sum(np.power(B@v, 2))
        subgrad = lambda v: g + B.T@(B@v)
        res = minimize(subprob, x, jac=subgrad, method="CG", \
                      options={'gtol': 1e-3, 'norm': 2.0, 'eps': 0.1, \
                                       'maxiter': None, 'disp': False})
        dx = res.x
        
        decr = np.sum(dx*g)
        
        #return if epsilon suboptimal
        if (decr**2) <= 2*tol:
            xstar = x
            opt_gaps = np.apply_along_axis(f, 0,  (xs[:, :numiter+1])) - f(xstar)
            return xstar, opt_gaps, times[:,:numiter+1], numiter, xs
        
        #line search
        u = 0.2
        fval = f(x)
        while f(x + u*dx) > fval + 0.25*u*decr:
            u = 0.95*u
        #print(u)
        # update and keep track of everything
        times[:,i+1] = time.time() - t0
        x = x + u*dx
        xs[:,i+1] = x
        numiter += 1
        
    xstar = x
    opt_gaps = np.apply_along_axis(f, 0,  (xs[:, :numiter+1])) - f(xstar)
    return xstar, opt_gaps, times[:,:numiter+1], numiter, xs
        
    
    

## Newton's Method

In [ ]:
def Newton(f, grad, Hess_sr, x0):
    # algorithm parameters
    tol = 1e-6
    maxiter = 1000

    
    #initialize quantities and note sizes of quantities
    d = x0.size
    H_sr = Hess_sr(x0);
    n = H_sr.shape[0]
        
    #solve the newton-sketch problem
    dx = 100 #dummy value to start iterating
    xs = np.zeros((x0.size, maxiter+1))
    xs[:,0] = x0
    t0 = time.time()
    times = np.zeros((1, maxiter+1))
    numiter = 0
    x = x0
    for i in range(maxiter):
        #print(numiter)
        
        #find the approximate newton direction
        H_sr = Hess_sr(x);
        g = grad(x)
        
        subprob = lambda v: g.T@v + 0.5*np.sum(np.power(H_sr@v, 2))
        subgrad = lambda v: g + H_sr.T@(H_sr@v)
        res = minimize(subprob, x, jac=subgrad, method="BFGS", \
                      options={'gtol': 1e-3, 'norm': 2.0, 'eps': 0.1, \
                                       'maxiter': None, 'disp': False})
        dx = res.x
        
        decr = np.sum(dx*g)
        
        #return if epsilon suboptimal
        if (decr**2) <= 2*tol:
            xstar = x
            opt_gaps = np.apply_along_axis(f, 0,  (xs[:, :numiter+1])) - f(xstar)
            return xstar, opt_gaps, times[:,:numiter+1], numiter, xs
        
        #line search
        u = 1
        fval = f(x)
        while f(x + u*dx) > fval + 0.25*u*decr:
            u = 0.95*u
        #print(u)
        # update and keep track of everything
        times[:,i+1] = time.time() - t0
        x = x + u*dx
        xs[:,i+1] = x
        numiter += 1
        
    xstar = x
    opt_gaps = np.apply_along_axis(f, 0,  (xs[:, :numiter+1])) - f(xstar)
    return xstar, opt_gaps, times[:,:numiter+1], numiter, xs
        

## Gradient Descent

In [ ]:
def gd(f, grad, x0):
    # algorithm parameters
    tol = 1e-5
    maxiter = 1000
    
    #initialize quantities and note sizes of quantities
    d = x0.size
        
    #solve the newton-sketch problem
    dx = 100 #dummy value to start iterating
    xs = np.zeros((x0.size, maxiter+1))
    xs[:,0] = x0
    t0 = time.time()
    times = np.zeros((1, maxiter+1))
    numiter = 0
    x = x0
    for i in range(maxiter):
        #print(numiter)
        
        #find the descent direction
        g = grad(x)
        dx = -g
        
        #return if epsilon suboptimal
        if np.sum(np.power(dx,2)) <= tol:
            xstar = x
            opt_gaps = np.apply_along_axis(f, 0,  (xs[:, :numiter+1])) - f(xstar)
            return xstar, opt_gaps, times[:,:numiter+1], numiter, xs
        
        #line search
        u = 1
        fval = f(x)
        while f(x + u*dx) > fval + 0.25*u*np.dot(dx,g):
            u = 0.95*u
        #print(u)
        # update and keep track of everything
        times[:,i+1] = time.time() - t0
        x = x + u*dx
        xs[:,i+1] = x
        numiter += 1
        
    xstar = x
    opt_gaps = np.apply_along_axis(f, 0,  (xs[:, :numiter+1])) - f(xstar)
    return xstar, opt_gaps, times[:,:numiter+1], numiter, xs

## Least Squares Experiment

In [ ]:
#define cost, grad, hess square root for least squares
import numpy as np

n = 2000
d = 500
m = d

A = 3*np.random.rand(n, d)
b = A@np.random.rand(d) + 0.5*np.random.randn(n)
x0 = 5*np.ones(d)

#least squares cost
def ls(x):
    return np.sum(np.power(A@x-b, 2))

#least squares gradient
def lsgrad(x):
    return A.T@(A@x-b)

#least squares sqrt hessian
def lshess_sr(x):
    return A

x_opt, _, _, _ = np.linalg.lstsq(A, b, rcond=None)

In [ ]:
xstar_ns, gaps_ns, times_ns, numiter_ns, xs_ns = Newton_Sketch(ls, lsgrad, \
                                                               lshess_sr, x0, int(0.55*n), SketchType='SubSamp')
print(times_ns[:,-1], numiter_ns)

In [ ]:
xstar_nt, gaps_nt, times_nt, numiter_nt, xs_nt = Newton(ls, lsgrad, lshess_sr, x0)
print(times_nt[:,-1], numiter_nt)

In [ ]:
xstar_gd, opt_gaps_gd, times_gd, numiter_gd, xs_gd = gd(ls, lsgrad, x0)
print(times_gd[:,-1], numiter_gd)

In [ ]:
gaps_gd = np.apply_along_axis(ls, 0, xs_gd) - ls(x_opt)
import matplotlib.pyplot as plt
plt.plot( times_ns.flatten(), gaps_ns + 1e-5, 'r^',  times_nt.flatten(), gaps_nt+ 1e-5, 'bs', \
         times_gd.flatten()[:-1], opt_gaps_gd[:-1] + 1e-5, 'g--')

plt.yscale('log')
plt.legend(['Newton-Sketch', 'Newton', 'GD'])
plt.title('Optimality Gap vs Wall Clock Time')
plt.show()

In [ ]:
gaps_gd = np.apply_along_axis(ls, 0, xs_gd) - ls(x_opt)
import matplotlib.pyplot as plt
plt.plot( range(numiter_ns+1), gaps_ns + 1e-5, 'r^',  range(numiter_nt+1), gaps_nt+ 1e-5, 'bs', \
         range(numiter_gd), opt_gaps_gd[:-1] + 1e-5, 'g--')

plt.yscale('log')
plt.legend(['Newton-Sketch', 'Newton', 'GD'])
plt.title('Optimality Gap vs Iteration')
plt.show()

In [ ]:
#define cost, grad, hess square root for logistic regression
import numpy as np
import sklearn.linear_model as lm
import time
from scipy.optimize import *

n = 2000
d = 100
m = d

A = np.random.rand(n, d+1)
#A[int(n/2):, :] = np.random.rand(int(n/2), d+1) + 0.1
A[:, 0] = 1; #for intercept
b = np.round(np.sum(A + 0.9*np.random.rand(n, d+1) + 2, axis=1))
x0 = np.zeros(d+1)

#random convex cost
def ps(x):
    return np.sum(np.exp(b * (A@x)))

#least squares gradient
def psgrad(x):
    return np.sum((A.T * b * np.exp(b * (A@x))).T, axis=0)

#least squares sqrt hessian
def pshess_sr(x):
    return ((np.exp(b * (A@x))**0.5) * A.T).T


In [ ]:
xstar_ns, gaps_ns, times_ns, numiter_ns, xs_ns = Newton_Sketch(ps, psgrad, pshess_sr, \
                                                               x0, int(0.85*n), SketchType='SubSamp')
#xstar_ns, gaps_ns, times_ns, numiter_ns = Newton_Sketch(ps, psgrad, pshess_sr, x0, int(0.6*n), SketchType='Gaussian')
print(times_ns[:,-1], numiter_ns)

In [ ]:
xstar_nt, gaps_nt, times_nt, numiter_nt, xs_nt = Newton(ps, psgrad, pshess_sr, x0)
print(times_nt[:,-1], numiter_nt)

In [ ]:
xstar_gd, opt_gaps_gd, times_gd, numiter_gd, xs_gd = gd(ps, psgrad, x0)
print(times_gd[:,-1], numiter_gd)

In [ ]:
gaps_gd = np.apply_along_axis(ls, 0, xs_gd) - ls(xstar_nt)
import matplotlib.pyplot as plt
plt.plot( times_ns.flatten(), gaps_ns + 1e-5, 'r^',  times_nt.flatten(), gaps_nt+ 1e-5, 'bs', \
         times_gd.flatten()[:-1], opt_gaps_gd[:-1] + 1e-5, 'g--')

plt.yscale('log')
plt.legend(['Newton-Sketch', 'Newton', 'GD'])
plt.title('Optimality Gap vs Wall Clock Time')
plt.show()

In [ ]:
gaps_gd = np.apply_along_axis(ls, 0, xs_gd) - ls(xstar_nt)
import matplotlib.pyplot as plt
plt.plot( range(numiter_ns+1), gaps_ns + 1e-5, 'r^',  range(numiter_nt+1), gaps_nt+ 1e-5, 'bs', \
         range(numiter_gd), opt_gaps_gd[:-1] + 1e-5, 'g--')

plt.yscale('log')
plt.legend(['Newton-Sketch', 'Newton', 'GD'])
plt.title('Optimality Gap vs Iteration')
plt.show()